In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet


from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV

In [122]:
train = pd.read_csv('Data_Train.csv')

In [123]:
test = pd.read_csv('Data_Test.csv')

In [124]:
sample = pd.read_csv('Sample_Submission.csv')

In [125]:
sample_xl = pd.read_excel('Sample_Submission.xlsx')

In [126]:
sample_xl

,Unique_ID,Views
0,562546,7
1,907584,98
2,213013,31
3,340312,51
4,41854,85
...,...,...
19610,183403,96
19611,1272578,74
19612,539454,93
19613,1543013,56


In [127]:
train.head()

,Unique_ID,Name,Genre,Country,Song_Name,Timestamp,Views,Comments,Likes,Popularity,Followers
0,413890,Hardstyle,danceedm,AU,N-Vitral presents BOMBSQUAD - Poison Spitter (...,2018-03-30 15:24:45.000000,14017,4,499,97,119563
1,249453,Dj Aladdin,danceedm,AU,Dj Aladdin - Old School Hip Hop Quick Mix,2016-06-20 05:58:52.000000,1918,17,49,17,2141
2,681116,Maxximize On Air,danceedm,AU,Maxximize On Air - Mixed by Blasterjaxx - Epis...,2015-05-08 17:45:59.000000,9668,11,312,91,22248
3,387253,GR6 EXPLODE,rbsoul,AU,MC Yago - Tenho Compromisso (DJ R7),2017-06-08 23:50:03.000000,113036,2,"2,400",76,393655
4,1428029,Tritonal,danceedm,AU,Escape (feat. Steph Jones),2016-09-17 20:50:19.000000,110024,81,"3,031",699,201030


In [128]:
test.head()

,Unique_ID,Name,Genre,Country,Song_Name,Timestamp,Comments,Likes,Popularity,Followers
0,562546,L2Share♫79,all-music,AU,엔플라잉 N.Flying - Rooftop (옥탑방),2019-01-02 09:49:19.000000,26,"5,428",118,2568
1,907584,Morgan Page,danceedm,AU,Morgan Page - In The Air - Episode 246,2015-03-09 17:30:10.000000,18,738,195,155645
2,213013,Dirty Workz,danceedm,AU,Outlander - Devotion (Official HQ Preview),2013-09-18 16:09:03.000000,18,386,96,29446
3,340312,Gucci Mane,all-music,AU,Freaky Gurl,2007-08-14 00:00:00.000000,1,388,19,799410
4,41854,Angga Kecap,rock,AU,Radical Nusantara - Teman,2018-08-10 16:28:54.000000,0,79,1,2


In [129]:
train.columns

Index(['Unique_ID', 'Name', 'Genre', 'Country', 'Song_Name', 'Timestamp',
       'Views', 'Comments', 'Likes', 'Popularity', 'Followers'],
      dtype='object')

In [130]:
test.columns

Index(['Unique_ID', 'Name', 'Genre', 'Country', 'Song_Name', 'Timestamp',
       'Comments', 'Likes', 'Popularity', 'Followers'],
      dtype='object')

In [131]:
train.shape

(78458, 11)

In [132]:
test.shape

(19615, 10)

In [133]:
df = pd.concat([train,test], axis='index')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [102]:
df.describe(include = 'all')

,Comments,Country,Followers,Genre,Likes,Name,Popularity,Song_Name,Timestamp,Unique_ID,Views
count,98073.000000,98073,9.807300e+04,98073,98073,98073,98073,98072,98073,9.807300e+04,7.845800e+04
unique,NaN,1,NaN,21,9758,1219,6259,98072,84593,NaN,NaN
top,NaN,AU,NaN,danceedm,0,Mixmag,0,ARMNHMR ft. Desiree Dawson - Fallen (William B...,2011-01-01 00:00:00.000000,NaN,NaN
freq,NaN,98073,NaN,28319,2355,2598,8057,1,346,NaN,NaN
mean,115.987591,NaN,4.796740e+05,NaN,NaN,NaN,NaN,NaN,NaN,7.458773e+05,5.469686e+05
std,756.416793,NaN,1.168391e+06,NaN,NaN,NaN,NaN,NaN,NaN,4.807398e+05,3.883085e+06
min,0.000000,NaN,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
25%,1.000000,NaN,1.778400e+04,NaN,NaN,NaN,NaN,NaN,NaN,3.205090e+05,6.216500e+03
50%,12.000000,NaN,8.892700e+04,NaN,NaN,NaN,NaN,NaN,NaN,6.618980e+05,2.538650e+04
75%,61.000000,NaN,3.936550e+05,NaN,NaN,NaN,NaN,NaN,NaN,1.176694e+06,1.201342e+05


In [103]:
df.duplicated().sum()

0

In [104]:
df.isna().sum()

Comments          0
Country           0
Followers         0
Genre             0
Likes             0
Name              0
Popularity        0
Song_Name         1
Timestamp         0
Unique_ID         0
Views         19615
dtype: int64

In [105]:
df['Comments'].nunique()

2379

In [106]:
df['Comments'].value_counts()

0       18979
1        6253
2        4237
3        3244
4        2764
        ...  
9821        1
1689        1
1753        1
5915        1
2388        1
Name: Comments, Length: 2379, dtype: int64

In [22]:
df['Country'].unique()

array(['AU'], dtype=object)

In [23]:
df['Country'].value_counts()

AU    98073
Name: Country, dtype: int64

In [24]:
df['Followers'].nunique()

1161

In [25]:
df['Followers'].value_counts()

1403057    2598
1007396    1773
393655     1747
119563     1422
87682      1229
           ... 
1633          1
24572         1
247           1
734           1
12367         1
Name: Followers, Length: 1161, dtype: int64

In [26]:
df['Genre'].value_counts()

danceedm                28319
electronic              22981
all-music               12839
rbsoul                   6802
latin                    5683
trap                     5018
rock                     4978
deephouse                3017
pop                      2254
dubstep                  1355
ambient                  1077
classical                 958
drumbass                  812
metal                     651
disco                     471
country                   414
alternativerock           186
hiphoprap                 129
reggaeton                  66
folksingersongwriter       51
indie                      12
Name: Genre, dtype: int64

In [27]:
df['Song_Name'].nunique()

98072

In [28]:
df['Song_Name'].isna().sum()

1

In [29]:
df['Unique_ID'].nunique()

98073

In [30]:
df['Name'].nunique()

1219

In [31]:
df['Popularity'].nunique()

6259

In [32]:
df['Timestamp'].nunique()

84593

In [33]:
df

,Comments,Country,Followers,Genre,Likes,Name,Popularity,Song_Name,Timestamp,Unique_ID,Views
0,4,AU,119563,danceedm,499,Hardstyle,97,N-Vitral presents BOMBSQUAD - Poison Spitter (...,2018-03-30 15:24:45.000000,413890,14017.0
1,17,AU,2141,danceedm,49,Dj Aladdin,17,Dj Aladdin - Old School Hip Hop Quick Mix,2016-06-20 05:58:52.000000,249453,1918.0
2,11,AU,22248,danceedm,312,Maxximize On Air,91,Maxximize On Air - Mixed by Blasterjaxx - Epis...,2015-05-08 17:45:59.000000,681116,9668.0
3,2,AU,393655,rbsoul,"2,400",GR6 EXPLODE,76,MC Yago - Tenho Compromisso (DJ R7),2017-06-08 23:50:03.000000,387253,113036.0
4,81,AU,201030,danceedm,"3,031",Tritonal,699,Escape (feat. Steph Jones),2016-09-17 20:50:19.000000,1428029,110024.0
...,...,...,...,...,...,...,...,...,...,...,...
19610,17,AU,357007,danceedm,"1,775",Deep Sounds,341,Jordan Burns - Your Mind,2017-08-29 20:39:57.000000,183403,NaN
19611,12,AU,17356,danceedm,"2,095",Sjammienators,305,Sjammienators & System Overload & MC Komplex -...,2016-01-22 13:51:52.000000,1272578,NaN
19612,5,AU,165268,electronic,39,Kompakt,12,Vermont - Paradigma (Wassermann Krautramatrik ...,2017-10-27 04:51:20.000000,539454,NaN
19613,113,AU,52573,electronic,"3,754",Yheti,666,Yheti & Metapod - Ominous Combo,2017-08-30 16:35:20.000000,1543013,NaN


In [71]:
#df['Timestamp'] = max(df['Timestamp']) - df['Timestamp']

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [134]:
df.drop(['Song_Name', 'Timestamp'], axis=1, inplace=True)

In [108]:
df

,Comments,Country,Followers,Genre,Likes,Name,Popularity,Views
0,4,AU,119563,danceedm,499,Hardstyle,97,14017.0
1,17,AU,2141,danceedm,49,Dj Aladdin,17,1918.0
2,11,AU,22248,danceedm,312,Maxximize On Air,91,9668.0
3,2,AU,393655,rbsoul,"2,400",GR6 EXPLODE,76,113036.0
4,81,AU,201030,danceedm,"3,031",Tritonal,699,110024.0
...,...,...,...,...,...,...,...,...
19610,17,AU,357007,danceedm,"1,775",Deep Sounds,341,NaN
19611,12,AU,17356,danceedm,"2,095",Sjammienators,305,NaN
19612,5,AU,165268,electronic,39,Kompakt,12,NaN
19613,113,AU,52573,electronic,"3,754",Yheti,666,NaN


In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98073 entries, 0 to 19614
Data columns (total 8 columns):
Comments      98073 non-null int64
Country       98073 non-null object
Followers     98073 non-null int64
Genre         98073 non-null object
Likes         98073 non-null object
Name          98073 non-null object
Popularity    98073 non-null object
Views         78458 non-null float64
dtypes: float64(1), int64(2), object(5)
memory usage: 6.7+ MB


In [110]:
#df['Likes']=df.Likes.str.replace(',','.','k', '').astype(float)

In [111]:
df['Likes']

0          499
1           49
2          312
3        2,400
4        3,031
         ...  
19610    1,775
19611    2,095
19612       39
19613    3,754
19614      627
Name: Likes, Length: 98073, dtype: object

In [135]:
cat_column = df.select_dtypes(exclude=np.number).columns

In [ ]:
df

In [136]:
num_column = df.select_dtypes(include=np.number).columns
num_column

Index(['Comments', 'Followers', 'Unique_ID', 'Views'], dtype='object')

In [137]:
cat_column

Index(['Country', 'Genre', 'Likes', 'Name', 'Popularity'], dtype='object')

df_cat_dummy = pd.get_dummies(df[cat_column].drop(df.columns[["Song_Name", "Timestamp"]], axis='1'))

In [138]:
df_cat_dummy = pd.get_dummies(df[cat_column])

In [139]:
df_preprocessed = pd.concat([df[num_column],df_cat_dummy,df["Unique_ID"]]
         ,axis = "columns")

In [140]:
df_preprocessed["Views"].isna().sum()

19615

In [141]:
df_preprocessed["Unique_ID"].isna().sum()

Unique_ID    0
Unique_ID    0
dtype: int64

In [46]:
df_preprocessed["Views"].isna()

0        False
1        False
2        False
3        False
4        False
         ...  
19610     True
19611     True
19612     True
19613     True
19614     True
Name: Views, Length: 98073, dtype: bool

In [142]:
test_processed = df_preprocessed[df_preprocessed["Views"].isna() == False]

In [143]:
test_processed.shape

(78458, 17263)

In [144]:
train_processed = df_preprocessed[df_preprocessed["Views"].notna() == False]

In [145]:
train_processed.shape

(19615, 17263)

In [146]:
y = train_processed['Views']

In [147]:
x = train_processed.drop(columns=['Unique_ID', 'Views'])

In [148]:
test_x = test_processed.drop(columns=['Unique_ID', 'Views'])

In [149]:
linear = LinearRegression()
linear.fit(x,y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [150]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19615 entries, 0 to 19614
Columns: 17260 entries, Comments to Popularity_999
dtypes: int64(2), uint8(17258)
memory usage: 323.3 MB


In [153]:
x.columns

Index(['Comments', 'Followers', 'Country_AU', 'Genre_all-music',
       'Genre_alternativerock', 'Genre_ambient', 'Genre_classical',
       'Genre_country', 'Genre_danceedm', 'Genre_deephouse',
       ...
       'Popularity_990', 'Popularity_991', 'Popularity_992', 'Popularity_993',
       'Popularity_994', 'Popularity_995', 'Popularity_996', 'Popularity_997',
       'Popularity_998', 'Popularity_999'],
      dtype='object', length=17260)